In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
IMG_PATH="../facial_expressions/"

In [4]:
emotions=["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [5]:
df=pd.read_csv(IMG_PATH+"icml_face_data.csv")

In [6]:
df=df.replace(to_replace="PrivateTest",value="Testing")
df=df.replace(to_replace="PublicTest",value="Training")

In [7]:
df

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,6,Testing,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,3,Testing,178 174 172 173 181 188 191 194 196 199 200 20...
35884,0,Testing,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,3,Testing,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [8]:
face_dim=64

In [9]:
import matplotlib.pyplot as plt

In [10]:
def ret_keypoints(image):
    gray=image
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    param=[0,0,face_dim,face_dim]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [11]:
data={"Training":[],"Testing":[]}

In [12]:
# for i in range(len(df)):
#     print("\r",i+1,end=" ")
#     row=df.iloc[i]
#     img=np.array(list(map(np.uint8,row[2].split()))).reshape(48,48)
#     data[row[1]].append([row[0],ret_keypoints(img)])

In [13]:
import pickle

In [14]:
# with open("data.pkl","wb") as f:
#     pickle.dump(data, f)

In [15]:
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)

In [16]:
# s=np.arange(len(XXD))
# for i in range(5):
#     np.random.shuffle(s)
#     XXD=XXD[s]
#     YYD=YYD[s]

In [17]:
import sys
sys.path.append("../dnn_from_scratch")

In [18]:
from nnet.network import Sequential
from nnet.layers import dense,dropout,BatchNormalization
from nnet import functions
from nnet import optimizers

Seed: 347
Seed: 177


In [19]:
model=Sequential()
model.add(dense(1024,activation=functions.relu, input_shape=(68*2)))
model.add(BatchNormalization())
model.add(dropout(0.4))
model.add(dense(512,activation=functions.relu))
model.add(BatchNormalization())
model.add(dropout(0.3))
model.add(dense(256,activation=functions.relu))
model.add(BatchNormalization())
model.add(dropout(0.2))
model.add(dense(128,activation=functions.relu))
model.add(BatchNormalization())
model.add(dropout(0.1))
model.add(dense(len(emotions),activation=functions.softmax))

In [20]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
0 input_layer(InputLayer) (None, 136)                echo             0
__________________________________________________________________________________________
1 dense(dense)            (None, 1024)               relu             140288
__________________________________________________________________________________________
2 BatchNormalization(Batc (None, 1024)               echo             4096
__________________________________________________________________________________________
3 dropout(dropout)        (None, 1024)               echo             0
__________________________________________________________________________________________
4 dense(dense)            (None, 512)                relu             524800
__________________________________________________________________________________________
5 B

In [21]:
model.compile(optimizer=optimizers.adam,loss=functions.cross_entropy_with_logits)

In [25]:
XDB=np.vstack(np.asarray(data["Training"])[:,1]).astype(np.float32)
YDY=np.asarray(data["Training"])[:,0]
YDB=np.zeros((len(YDY),len(emotions)),dtype=np.float32)
for i in range(len(YDB)):
    YDB[i][YDY[i]]=1
XTB=np.vstack(np.asarray(data["Testing"])[:,1]).astype(np.float32)
YTY=np.asarray(data["Testing"])[:,0]
YTB=np.zeros((len(YTY),len(emotions)),dtype=np.float32)
for i in range(len(YTB)):
    YTB[i][YTY[i]]=1

In [26]:
with open('extra_data.pkl', 'rb') as f:
    exD,eyD = pickle.load(f)

In [27]:
XDB=np.concatenate((XDB,exD))
YDB=np.concatenate((YDB,eyD))

In [28]:
s=np.arange(len(XDB))
for i in range(5):
    np.random.shuffle(s)
    XDB=XDB[s]
    YDB=YDB[s]

In [29]:
model.fit(XDB,YDB,batch_size=64,epochs=100,validation_data=(XTB,YTB))

EPOCH: 1 / 100
Progress:  32960 / 32955  - 0s - 0.016s/sample - loss: 1.2220 - accuracy: 0.5008      ..      .      .      .
Epoch time: 9.710
Validation Accuracy: 0.367 - val_loss: 0.7474
EPOCH: 2 / 100
Progress:  32960 / 32955  - 0s - 0.021s/sample - loss: 1.2499 - accuracy: 0.4209      ..      .
Epoch time: 10.619
Validation Accuracy: 0.426 - val_loss: 0.5533
EPOCH: 3 / 100
Progress:  32960 / 32955  - 0s - 0.021s/sample - loss: 1.1470 - accuracy: 0.4849      ..      .
Epoch time: 12.712
Validation Accuracy: 0.465 - val_loss: 0.4847
EPOCH: 4 / 100
Progress:  32960 / 32955  - 0s - 0.021s/sample - loss: 0.9408 - accuracy: 0.5033      ..      .      .      .      .      .      .
Epoch time: 12.801
Validation Accuracy: 0.460 - val_loss: 0.4900
EPOCH: 5 / 100
Progress:  32960 / 32955  - 0s - 0.021s/sample - loss: 1.1422 - accuracy: 0.4872      ..      .      .
Epoch time: 13.214
Validation Accuracy: 0.435 - val_loss: 0.5231
EPOCH: 6 / 100
Progress:  32960 / 32955  - 0s - 0.042s/sample - l

Progress:  32960 / 32955  - 0s - 0.022s/sample - loss: 1.0792 - accuracy: 0.5493      ..
Epoch time: 12.761
Validation Accuracy: 0.444 - val_loss: 0.5739
EPOCH: 45 / 100
Progress:  32960 / 32955  - 0s - 0.022s/sample - loss: 1.0480 - accuracy: 0.4978      ..      .      .      .      .      .
Epoch time: 12.710
Validation Accuracy: 0.495 - val_loss: 0.5083
EPOCH: 46 / 100
Progress:  32960 / 32955  - 0s - 0.022s/sample - loss: 0.9908 - accuracy: 0.4817      ..      .      .      .      .
Epoch time: 12.683
Validation Accuracy: 0.448 - val_loss: 0.5622
EPOCH: 47 / 100
Progress:  32960 / 32955  - 0s - 0.022s/sample - loss: 0.9035 - accuracy: 0.6141      ..      .
Epoch time: 13.429
Validation Accuracy: 0.500 - val_loss: 0.5274
EPOCH: 48 / 100
Progress:  32960 / 32955  - 0s - 0.021s/sample - loss: 0.8783 - accuracy: 0.5828      ..      .      .      .      .
Epoch time: 12.776
Validation Accuracy: 0.495 - val_loss: 0.5497
EPOCH: 49 / 100
Progress:  32960 / 32955  - 0s - 0.039s/sample - los

Progress:  32960 / 32955  - 0s - 0.022s/sample - loss: 0.8833 - accuracy: 0.5871      ..
Epoch time: 12.718
Validation Accuracy: 0.488 - val_loss: 0.6052
EPOCH: 91 / 100
Progress:  32960 / 32955  - 0s - 0.024s/sample - loss: 0.9198 - accuracy: 0.6058      ..
Epoch time: 13.054
Validation Accuracy: 0.497 - val_loss: 0.6065
EPOCH: 92 / 100
Progress:  32960 / 32955  - 0s - 0.023s/sample - loss: 1.0649 - accuracy: 0.5500      ..      .      .
Epoch time: 12.563
Validation Accuracy: 0.504 - val_loss: 0.5862
EPOCH: 93 / 100
Progress:  32960 / 32955  - 0s - 0.023s/sample - loss: 0.8184 - accuracy: 0.6383      ..
Epoch time: 12.569
Validation Accuracy: 0.466 - val_loss: 0.6319
EPOCH: 94 / 100
Progress:  32960 / 32955  - 0s - 0.023s/sample - loss: 0.7808 - accuracy: 0.6226      ..
Epoch time: 12.721
Validation Accuracy: 0.474 - val_loss: 0.7063
EPOCH: 95 / 100
Progress:  32960 / 32955  - 0s - 0.023s/sample - loss: 0.8051 - accuracy: 0.5844      ..
Epoch time: 12.861
Validation Accuracy: 0.499 -

In [76]:
model.fit(XDB,YDB,batch_size=64,epochs=100,validation_data=(XTB,YTB))

Train on 32955 samples, validate on 3589 samples
Epoch 1/100
32955/32955 [==============================] - 10s 307us/sample - loss: 1.6506 - accuracy: 0.3841 - val_loss: 1.7176 - val_accuracy: 0.3126
Epoch 2/100
32955/32955 [==============================] - 8s 255us/sample - loss: 1.4670 - accuracy: 0.4359 - val_loss: 1.7464 - val_accuracy: 0.3110
Epoch 3/100
32955/32955 [==============================] - 8s 245us/sample - loss: 1.4218 - accuracy: 0.4525 - val_loss: 1.6110 - val_accuracy: 0.3344
Epoch 4/100
32955/32955 [==============================] - 10s 291us/sample - loss: 1.3990 - accuracy: 0.4623 - val_loss: 1.4686 - val_accuracy: 0.4118
Epoch 5/100
32955/32955 [==============================] - 8s 242us/sample - loss: 1.3825 - accuracy: 0.4707 - val_loss: 1.5474 - val_accuracy: 0.3826
Epoch 6/100
32955/32955 [==============================] - 8s 228us/sample - loss: 1.3754 - accuracy: 0.4696 - val_loss: 1.6248 - val_accuracy: 0.3605
Epoch 7/100
32955/32955 [==================

In [313]:
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("model.h5")

In [99]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

In [ ]:
model=loaded_model

In [31]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.48202842017275005

In [32]:
model.predict(XTB).argmax(axis=1)

array([0, 4, 4, ..., 4, 2, 6])

In [33]:
YTB.argmax(axis=1)

array([0, 5, 6, ..., 0, 3, 2])

In [34]:
from udp_streamer import *

In [93]:
handler = udp_handler()
handler.make_listener('0.0.0.0',5555)

In [35]:
from mtcnn import MTCNN
detector = MTCNN()

Using TensorFlow backend.


In [36]:
def face_rectangles(rgb):
    faces = detector.detect_faces(rgb)
    all_recs = []
    for x in faces:
        rec = x['box']
        all_recs.append(dlib.rectangle(left=rec[0], top=rec[1], right=rec[0]+rec[2], bottom=rec[1]+rec[3]))
    return all_recs

In [37]:
def get_img():
    global handler,img
    buffer = handler.get_data()
    if buffer is not None:
        try:
            npimg = np.frombuffer(buffer, dtype=np.uint8)
            img = cv2.imdecode(npimg, 1)
        except Exception as e:
            print(e)
    return img

In [38]:
cam = cv2.VideoCapture(0)

while True:
    ret, img = cam.read()
#     img=get_img()
    try:
        grimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except:
        continue
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     faces = face_cascade.detectMultiScale(gimg, 1.3, 5)
    faces = face_rectangles(gimg)
#     for (x,y,w,h) in faces:
    for facrec in faces:
        x=facrec.left()
        y=facrec.top()
        w=facrec.right()-x
        h=facrec.bottom()-y
        landmarks=[]
        svx,svy=x,y
        gray=grimg[y:y+int(1.1*h),max(0,x-int(0.15*w)):x+int(1.15*w)]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 1, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
        if len(landmarks)>0:
            y_out=model.predict(np.asarray(landmarks))
            res=np.argmax(y_out,axis=1)
            for r in res:
                cv2.putText(img,emotions[r],(svx,svy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

cam.release()
cv2.destroyAllWindows()

In [35]:
cam.release()
cv2.destroyAllWindows()